In [ ]:
!pip install fastai --upgrade

In [ ]:
import pandas as pd
from fastai.vision.all import *
import torchvision

In [ ]:
path = Path('../input/sorghum-id-fgvc-9')

In [ ]:
train_df = pd.read_csv(path/'train_cultivar_mapping.csv')

In [ ]:
train_df.describe()

Dropping NAN values

In [ ]:
train_df.dropna(inplace=True)

In [ ]:
train_df.shape

In [ ]:
train_df.describe()

In [ ]:
train_df.head()

Getting the unique values of cultivar and their counts

In [ ]:
train_df.cultivar.unique()

In [ ]:
train_df.cultivar.value_counts().reset_index()

In [ ]:
train_df = train_df.reset_index(drop=True)

In [ ]:
train_df.iloc[3329]

In [ ]:
train_df.head()

# Model creation and Training

Creating DataLoader from dataframe

In [ ]:
datablock = DataBlock(
    blocks = (ImageBlock, CategoryBlock),
    get_x = ColReader('image', path/'train_images'),
    get_y = ColReader('cultivar'),
    splitter = RandomSplitter(valid_pct=0.2),#splitter,
    item_tfms = Resize(225)
)

In [ ]:
dls = datablock.dataloaders(train_df, bs=32)

In [ ]:
learn = vision_learner(dls, torchvision.models.densenet201, metrics = accuracy)

Training the model

In [ ]:
#learn.fit(50, cbs=EarlyStoppingCallback(monitor='accuracy', patience=5, min_delta=0.001))
learn.fit(50)

In [ ]:
learn.export('./densenet_model.pkl')

# Inference

Inference for a single image

In [ ]:
#learn = load_learner('./densenet_model.pkl')

In [ ]:
learn.predict('../input/sorghum-id-fgvc-9/test/1000005362.png')

**Inference on complete data**

In [ ]:
test_images = get_image_files(path/'test')
test_dataloader = learn.dls.test_dl(test_images)

In [ ]:
test_images[0]

In [ ]:
preds, _ = learn.get_preds(dl=test_dataloader)

In [ ]:
preds[0]

In [ ]:
max(preds[0])

In [ ]:
preds.shape

Finding the class having maximum probability

In [ ]:
class_idxs = torch.argmax(preds, axis=1)
class_idxs.shape

In [ ]:
results = [dls.vocab[i] for i in class_idxs]

In [ ]:
len(class_idxs)

In [ ]:
len(test_images)

In [ ]:
len(results)

In [ ]:
class_idxs[:5]

In [ ]:
results[:5]

Creating a dataframe containg predictions for each image

In [ ]:
images = [img.name for img in (path/'test').ls()]
submissions = pd.DataFrame(list(zip(images, results)), columns = ['filename', 'cultivar'])

In [ ]:
submissions.head()

Saving the dataframe in the csv format

In [ ]:
submissions.to_csv('submission.csv', index = False)